<a href="https://colab.research.google.com/github/AnnSenina/Python_DH_MNE/blob/main/notebooks/Python_9_%D0%B1%D0%B8%D0%B1%D0%BB%D0%B8%D0%BE%D1%82%D0%B5%D0%BA%D0%B8_%D0%B4%D0%BB%D1%8F_DH_%D1%82%D0%B5%D0%BA%D1%81%D1%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Анализ текстов

In [ ]:
# все установки библиотек
!pip install wordcloud
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install eng-spacysentiment
!pip install textblob
!pip install dostoevsky
!python -m dostoevsky download fasttext-social-network-model

In [ ]:
# для очистки (препроцессинга) текста...
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')
ru_stop_words = stopwords.words('russian') # давайте для русского сохранять в отдельную переменную

# стемминг
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
ru_stemmer = SnowballStemmer("russian") # с русским работает плохл

# spaCy лемматизация английского
nlp = spacy.load("en_core_web_sm")

# 1 модель сентимент-анализа из коробки от spacy
import spacy
import eng_spacysentiment
# альтернатива - сентимент-анализ с TextBlob
from textblob import TextBlob
# сентимент-анализ текстов на русском
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel
tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

#таблички
import pandas as pd

# для рисования облака слов
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# для подсчета любых частот (часто встречающиеся слова, n-граммы и т.д.)
from collections import Counter

In [ ]:
# русский язык не лемматизируется NLTK - берем другие библиотеки

# pymorphy
!pip install pymorphy3 # недавно попробовала 3 версию, но и во 2 все равботает аналогично
from pymorphy3 import MorphAnalyzer
morph = MorphAnalyzer()

# mystem от Яндекса
!pip install pymystem3
from pymystem3 import Mystem
mystem = Mystem()

### Документация:

* [NLTK](https://www.nltk.org/index.html)
* [pymorphy](https://pymorphy2.readthedocs.io/en/stable/), таблица граммем
* [mystem](https://yandex.ru/dev/mystem/)
* [wordcloud](https://github.com/amueller/word_cloud)
* [spacy](https://github.com/explosion/spaCy), [сентимент-анализ](https://spacy.io/universe/project/eng_spacysentiment) в spacy
* [TextBlob](https://textblob.readthedocs.io/en/dev/)
* [dostoevsky](https://github.com/bureaucratic-labs/dostoevsky)

# Начнем с английского

In [ ]:
with open('01 - The Fellowship Of The Ring.txt', 'r', encoding='utf-8') as f:
  text = f.read()
text

In [ ]:
text = text.replace('\ufeff', '')
text = text.lower() # первый шаг - перевод к нижнему регистру
text

In [ ]:
# в зависимости от задач можно разделить текст сразу на токены
# или на предложения, а затем на токены

In [ ]:
text_tokens = word_tokenize(text) #NLTK
text_tokens # список слов

In [ ]:
text_sent = sent_tokenize(text)
print(text_sent) # список предложений
text_sent_tokens = []
for i in text_sent:
  l = word_tokenize(i)
  text_sent_tokens.append(l)
text_sent_tokens # список списков: предложения и слова

In [ ]:
# в токенах осталась пункуация с топ слова
stop_words # список стоп-слов из NLTK
# при необходимости его можно расширять так:
# stop_words = stop_words + ['ваше_слово1', 'ваше_слово2'...]

In [ ]:
clean_tokens = [] # чистим список токенов
for i in text_tokens:
  if i[0].isalpha() and i not in stop_words:
    clean_tokens.append(i)
clean_tokens

In [ ]:
# Давайте два разных способа очистки объединим в функции
def get_clean_tokens(text):
  text = text.lower()
  text_tokens = word_tokenize(text)
  clean_tokens = [] # чистим список токенов
  for i in text_tokens:
    if i[0].isalpha() and i not in stop_words:
      clean_tokens.append(i)
  return clean_tokens

example = get_clean_tokens(text)
example
# теперь вы можете отправлять туда текст и получать список токенов

In [ ]:
# не будем писать отдельную функцию - просто разделим на предложения и получим токены
example_2 = []
for i in sent_tokenize(text):
  s = get_clean_tokens(i)
  if len(s) > 0:
    example_2.append(s)
example_2

# зачем нам предложения? Пока оставим их, они пригодятся)

Для русского можно использовать такую же функцию, но передать стоп-слова на русском!

## Токены чистые - давайте стеммить и лемматизировать

In [ ]:
def get_stem_tokens(text_list):
  text_stemmed = []
  for i in text_list:
    text_stemmed.append(stemmer.stem(i))
  return text_stemmed

get_stem_tokens(example) # для русского аналогично - но результат будет хуже

Лемматизация - гораздо важнее и интереснее

In [ ]:
def get_lemmas(text_list):
  spacy_token = []
  doc = nlp(' '.join(text_list))
  for token in doc:
    if token.lemma_ != '-' and token.lemma_ != '.':
        spacy_token.append(token.lemma_)
  return spacy_token

example_lem = get_lemmas(example)
example_lem

In [ ]:
example_lem_2 = []
for i in example_2:
  s = get_lemmas(i)
  if len(s) > 0:
    example_lem_2.append(s)
example_lem_2
# давайте для одного предложения проведем сентимент-анализ

In [ ]:
df = pd.DataFrame(example_lem_2[0], columns=['token'])
df

In [ ]:
# spacy
nlp = eng_spacysentiment.load()

spacy_pos = []
spacy_neg = []
for i in df.token:
  doc = nlp(i)
  spacy_pos.append(round(doc.cats['positive'], 5))
  spacy_neg.append(round(doc.cats['negative'], 5))

df['spacy_pos'] = spacy_pos
df['spacy_neg'] = spacy_neg
df

In [ ]:
# теперь попробуем другой бибилиотекой!
# у нее другие параметры: оценка настроения + оценка на субъективность

blob_polar = []
blob_subj = []
for i in df.token:
  analysis = TextBlob(i).sentiment
  blob_polar.append(round(analysis[0], 5))
  blob_subj.append(round(analysis[1], 5))

df['blob_polar'] = blob_polar
df['blob_subj'] = blob_subj
df

Обычно так поступают с предложениями, а не отдельными словами, если текст большой

In [ ]:
sent = []
for i in example_lem_2:
  i = ' '.join(i)
  sent.append(i)
sent

In [ ]:
df2 = pd.DataFrame(sent, columns=['token'])
spacy_pos = []
spacy_neg = []
for i in df2.token:
  doc = nlp(i)
  spacy_pos.append(round(doc.cats['positive'], 5))
  spacy_neg.append(round(doc.cats['negative'], 5))

df2['spacy_pos'] = spacy_pos
df2['spacy_neg'] = spacy_neg

blob_polar = []
blob_subj = []
for i in df2.token:
  analysis = TextBlob(i).sentiment
  blob_polar.append(round(analysis[0], 5))
  blob_subj.append(round(analysis[1], 5))

df2['blob_polar'] = blob_polar
df2['blob_subj'] = blob_subj
df2

# Частоты

In [ ]:
Counter(example_lem).most_common(50)

##N-gramms
Текст можно разделить на n-граммы – устойчивые сочетания по N слов:

    nltk.bigrams() – сочетания по два слова
    nltk.trigrams() – сочетания по три слова
    nltk.ngrams(list, n) – сочетания по N слов

In [ ]:
freq_bigramms = Counter(nltk.bigrams(example_lem))
freq_bigramms.most_common(20)

In [ ]:
freq_ngramms = Counter(nltk.ngrams(example_lem, 4))
freq_nigramms.most_common(10)

# Облако слов

In [ ]:
# Генерируем облако слов
wordcloud = WordCloud().generate(', '.join(example_lem))
plt.imshow(wordcloud) # Что изображаем
plt.axis("off") # Без подписей на осях
plt.show() # показать изображение

In [ ]:
wordcloud = WordCloud(width = 2000,
                      height = 1500,
                      background_color='black',
                      colormap='Pastel1').generate(', '.join(example_lem))
plt.figure(figsize=(30, 15)) # Устанавливаем размер картинки
plt.imshow(wordcloud) # Что изображаем
plt.axis("off") # Без подписей на осях
plt.show() # показать изображение

# Попробуем с русским

Отличаться будет только лемматизация (NLTK не лемматизирует русский)


In [ ]:
# сначала простые примеры
morph.parse('человек')

In [ ]:
# сделаем красиво
print('Cлово - ', morph.parse('человеком')[0].word)
print('Лемма слова - ', morph.parse('человеком')[0].normal_form)
print('Грамматическая информация слова - ', morph.parse('человеком')[0].tag)
print('Часть речи слова - ', morph.parse('человеком')[0].tag.POS)
print('Род слова - ', morph.parse('человеком')[0].tag.gender)
print('Число слова - ', morph.parse('человеком')[0].tag.number)
print('Падеж слова - ', morph.parse('человеком')[0].tag.case)

In [ ]:
# для лемматизации нужна именно эта команда
morph.parse('человеком')[0].normal_form

# Важно!
pymorphy не умеет сам токенизировать

Пробуем на реальном тексте

In [ ]:
with open('Dostoevsky_PrestuplenieINakazanie.txt', 'r', encoding='utf-8') as f:
  text = f.read()
text

In [ ]:
def get_clean_tokens_ru(text):
  text = text.lower()
  text_tokens = word_tokenize(text)
  clean_tokens = [] # чистим список токенов
  for i in text_tokens:
    if i[0].isalpha() and i not in ru_stop_words: # отличие здесь!
      clean_tokens.append(i)
  return clean_tokens

clean_text_ru = get_clean_tokens_ru(text)
clean_text_ru

In [ ]:
ru_lemmatized_1 = []
for word in clean_text_ru:
    result = morph.parse(word)
    most_probable_result = result[0] ## почему мы берем первый разбор? см.в этом месте: https://pymorphy2.readthedocs.io/en/latest/user/guide.html#select-correct
    normal_form = most_probable_result.normal_form
    ru_lemmatized_1.append(normal_form)

ru_lemmatized_1

### MyStem

In [ ]:
# mystem лемматизирует сам
mystem.lemmatize('mystem даже сам умеет токенизировать текст')

In [ ]:
words_lemmatized_mystem = mystem.lemmatize(text.lower())
words_lemmatized_mystem

In [ ]:
words = []
for i in words_lemmatized_mystem:
  if i[0].isalpha() and i not in ru_stop_words:
    words.append(i)
words

### Остался сентимент-анализ

In [ ]:
results = model.predict(words[:30], k=2)

for word, sentiment in zip(words, results):
    print(word, '->', sentiment)

In [ ]:
# или по предложениям
example_ru = []
for i in sent_tokenize(text):
  s = get_clean_tokens_ru(i)
  if len(s) > 0:
    example_ru.append(s)
example_ru

In [ ]:
ru_lemmatized_sent = []
for s in example_ru:
  sent_lem = []
  for w in s:
    result = morph.parse(w)
    most_probable_result = result[0]
    normal_form = most_probable_result.normal_form
    sent_lem.append(normal_form)
  ru_lemmatized_sent.append(' '.join(sent_lem))

ru_lemmatized_sent

In [ ]:
results = model.predict(ru_lemmatized_sent[:20], k=2)

for sentence, sentiment in zip(ru_lemmatized_sent, results):
    print(sentence, '->', sentiment)

# Бонус:

так будет выглядеть функция, лемматизирующая английский / русский

In [ ]:
def get_en_lemm(text):
  text = text.lower()
  text_tokens = word_tokenize(text)
  clean_tokens = [] # чистим список токенов
  for i in text_tokens:
    if i[0].isalpha() and i not in stop_words:
      clean_tokens.append(i)
  spacy_token = []
  doc = nlp(' '.join(clean_tokens))
  for token in doc:
    if token.lemma_ != '-' and token.lemma_ != '.':
        spacy_token.append(token.lemma_)
  return spacy_token

# передаете текст на английском строкой, получаете список лемм
# пример:
some_text = '''There are very many things to enjoy in every season. Each year starts with joyful winter holidays. Big, white snowflakes are falling thick and fast and soon the ground, the roofs and the trees are covered with snow. Children and grown-ups may make snowmen, play snow balls, go skiing, skating or sliding down the snow covered hills.'''

print(get_en_lemm(some_text))

In [ ]:
def get_ru_lemm(text):
  text = text.lower()
  text_tokens = word_tokenize(text)
  clean_tokens = [] # чистим список токенов
  for i in text_tokens:
    if i[0].isalpha() and i not in ru_stop_words: # отличие здесь!
      clean_tokens.append(i)
  ru_lemmatized = []
  for word in clean_tokens:
      ru_lemmatized.append(morph.parse(word)[0].normal_form)
  return ru_lemmatized

some_text = """По результатам оценки критериев вузам присвоили один из рейтинговых статусов. Высшая школа экономики вошла в лидерскую группу А+, заняв первое место по таким критериям, как «Востребованность выпускников в найме», «Качество образовательной среды» и «Активность по развитию школьного образования».
Презентация рейтинга прошла в Москве на международной конференции AI Journey 2023. Один из основных критериев оценки вузов при расчете рейтинга — размер заработной платы молодых специалистов в течение года после завершения обучения (при трудоустройстве по специальности). В вузах-лидерах рейтинга средняя зарплата выпускников составила около 140 тыс. рублей.
В НИУ ВШЭ с 2021 года действует Центр искусственного интеллекта. Миссия Центра развитие и внедрение технологий искусственного интеллекта в разные сферы жизни человека и общества, отрасли науки и сектора экономики. НИУ ВШЭ стал одним из победителей конкурса на получение гранта от Правительства Российской Федерации для создания центра искусственного интеллекта.
Конкурс проводился в рамках федерального проекта «Искусственный интеллект», основная задача которого — стимулирование развития и внедрения технологий ИИ в России.
«По поручению главы государства Альянс в сфере ИИ совместно с Минобрнауки разработали рейтинг российских вузов по качеству подготовки специалистов по ИИ. Рейтинг является важнейшим индикатором качества образования в области ИИ и наглядно отражает мнение работодателей о том, насколько образовательные программы актуальны и отвечают запросу рынка. Среди 180 вузов, оказавшихся в рейтинге, 10 имеют оценки А+, А (хорошее качество) B+, B (приемлемое качество).
Абсолютные лидеры — это ВШЭ, МФТИ и ИТМО. Таким образом, ТОП-10 российских университетов уже могут конкурировать за звание лучших, а значит готовят высококвалифицированных специалистов и успешно развивают науку в области искусственного интеллекта», — отметил Дмитрий Чернышенко, заместитель председателя Правительства России, куратор нацпрограммы «Цифровая экономика».
Для объективной оценки качества специалистов по ИИ эксперты выделили 13 критериев, включая такие аспекты, как уровень заработных плат выпускников, их востребованность на рынке труда, наличие научных публикаций на конференциях и в журналах, количество призеров студенческих олимпиад, средний балл по ЕГЭ, и наличие программ, прошедших профессионально-общественную аккредитацию со стороны Альянса."""

print(get_ru_lemm(some_text))